Monitoramento e Fairness

In [ ]:
import pandas as pd 
import joblib
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

Carregar modelo e vetor já treinados

In [ ]:
modelo_path = '../model.joblib'
vetor_path = '../vectorizer.joblib'
dados_path = '../data/news_limpo.csv'

if not (os.path.exists(modelo_path) and os.path.exists(vetor_path)):
    raise FileExistsError('Treine e salve o modelo antes de executar!')

model = joblib.load(modelo_path)
vectorizer = joblib.load(vetor_path)
df = pd.read_csv(dados_path)
display(df.head())

Monitoramento

In [ ]:
novos_textos = ["Breaking news: New advancements in AI technology are transforming industries.",
                 "Exclusive report: Major breakthrough in renewable energy.",
                 "Update: New study reveals the impact of social media on mental health.",
                 "Breaking news: New study uncovers the effects of climate change on wildlife.",
                 "Exclusive report: Investigation into the rise of deepfake technology.",
                 # Fake
                 "Ovnis are real and have been visiting Earth for decades.",
                 "Aliens built the pyramids.",
                 "Bigfoot is hiding in the woods.",
                 "The Loch Ness Monster is real.",
                 "Mermaids have been spotted off the coast."
]

novos_df = pd.DataFrame({'title': novos_textos})
display(novos_df)

Vetorizar e predição

In [ ]:
novos_vetores = vectorizer.transform(novos_df['title'])
novos_preds = model.predict(novos_vetores)
novos_df['label_predito'] = novos_preds
print(novos_df)

Porcentagem de cada classe

In [ ]:
class_dist = novos_df['label_predito'].value_counts(normalize=True)
print('Distribuição dos rótulos preditos nos novos dados:')
print(class_dist)

Fairness - Vieses

In [ ]:
df['title_len'] = df['title'].apply(len)
df['len_category'] = pd.cut(df['title_len'], bins=[0,50,150,10000], labels=['curto', 'medio', 'longo'])
display(df['len_category'])


Predições conjunto de validação

In [ ]:
vetores = vectorizer.transform(df['title'])
df['pred'] = model.predict(vetores)
display(df['pred'])

Avaliação de acurácia

In [ ]:
for cat in df['len_category'].unique():
    subset = df[df['len_category'] == cat]
    if not subset.empty:
        acuracia = (subset['label'] == subset['pred']).mean()
        print(f'Acurácia para os textos {cat}: {acuracia:.2f} (N={len(subset)})')

Desafio fairness

In [ ]:
for cat in df['len_category'].unique():
    subset = df[df['len_category'] == cat]
    if not subset.empty:
        print(f'\nMatriz de confusão para textos {cat}:')
        print(confusion_matrix(subset['label'], subset["pred"]))
        sns.heatmap(confusion_matrix(subset['label'], subset["pred"]),
                    annot=True, fmt='d', cmap='Blues',
                     xticklabels=["Previsto Positivo", "Previsto Neutro", "Previsto Negativo"],
                    yticklabels=["Real Positivo", "Real Neutro", "Real Negativo"])
        plt.title(f'Textos {cat}')
        plt.xlabel('Predito')
        plt.ylabel('Real')
        plt.show()